In [1]:
# libraries
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
# our modules
from preprocessing import *

In [2]:
features_to_drop = ['Surface', 'WElo', 'WSurfElo', 'LElo', 'LSurfElo']

In [3]:
X, Y, playersElo = preprocess_data(2011, 2019, features_to_drop, returnElo=True)

In [4]:
# Build the model
rf = RandomForestClassifier(n_estimators=250,
                            bootstrap=True,
                            criterion='entropy',
                            max_features='log2',
                            max_depth=15,
                            n_jobs=-1)
rf.fit(X, Y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                       max_depth=15, max_features='log2', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=250,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [5]:
# TODO creare il dataset completo aggiungendo gli elo calcolati(WEloCalc, LEloCalc) e la ProbaElo
tournament = pd.read_csv("data/" + str(2020) + ".csv", encoding='utf-8-sig', dtype=DATA_TYPES)
elos = {'WEloCalc': [], 'LEloCalc': [], 'ProbaElo':[]}
for index, match in tournament.iterrows():
    elos['WEloCalc'].append(playersElo[match['Winner']])
    elos['LEloCalc'].append(playersElo[match['Loser']])
    elos['ProbaElo'].append(1 / (1 + 10 ** ((playersElo[match['Loser']] - playersElo[match['Winner']]) / 400)))
    print(match['Winner'], match['Loser'], playersElo[match['Winner']], playersElo[match['Loser']], 1 / (1 + 10 ** ((playersElo[match['Loser']] - playersElo[match['Winner']]) / 400)))

tournament = tournament.assign(WEloCalc=elos['WEloCalc'], LEloCalc=elos['LEloCalc'], ProbaElo=elos['ProbaElo'])
tournament

Bublik A. Mannarino A. 1629.6252062198644 1718.8668003078835 0.3743228341556116
Moutet C. Sandgren T. 1478.209082934734 1607.1359240030522 0.32253143346451085
Verdasco F. Andujar P. 1642.3690068942633 1606.5318397046785 0.5513916988628581
Bedene A. Ymer M. 1682.1801222335234 1541.371260773152 0.6922290293181144
Chardy J. Barrere G. 1708.2365104426692 1609.2546838485173 0.6387136087655984
Kecmanovic M. Thompson J. 1586.2353735716904 1649.7685062348069 0.4095741462197379
Krajinovic F. Edmund K. 1766.69586935994 1621.903554485132 0.6970926962310495
Wawrinka S. Chardy J. 1758.2938530316746 1708.2365104426692 0.5715439523554353
Herbert P.H. Cecchinato M. 1597.6574553312971 1465.306119021202 0.6817607941757897
Kukushkin M. Jaziri M. 1657.733637320562 1472.2039599937386 0.7442153711206722
Bedene A. Bublik A. 1682.1801222335234 1629.6252062198644 0.5750609810679478
Fucsovics M. Tiafoe F. 1576.1946323351071 1644.2162965141695 0.4033408142289658
Djere L. Sonego L. 1550.7793585808 1543.0060423028

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,...,WHand,WBHand,LElo,LSurfElo,LBD,LHand,LBHand,WEloCalc,LEloCalc,ProbaElo
0,1,Doha,Qatar Exxon Mobil Open,2020-01-06,ATP250,Outdoor,Hard,1st Round,3,Bublik A.,...,R,2.0,1782.0,1727.6,29/06/1988,L,2.0,1629.625206,1718.866800,0.374323
1,1,Doha,Qatar Exxon Mobil Open,2020-01-06,ATP250,Outdoor,Hard,1st Round,3,Moutet C.,...,L,2.0,1685.9,1626.7,22/07/1991,R,2.0,1478.209083,1607.135924,0.322531
2,1,Doha,Qatar Exxon Mobil Open,2020-01-06,ATP250,Outdoor,Hard,1st Round,3,Verdasco F.,...,L,2.0,1751.3,1622.6,23/01/1986,R,2.0,1642.369007,1606.531840,0.551392
3,1,Doha,Qatar Exxon Mobil Open,2020-01-06,ATP250,Outdoor,Hard,1st Round,3,Bedene A.,...,R,2.0,1826.2,1779.2,09/09/1998,R,2.0,1682.180122,1541.371261,0.692229
4,1,Doha,Qatar Exxon Mobil Open,2020-01-06,ATP250,Outdoor,Hard,1st Round,3,Chardy J.,...,R,2.0,1730.4,1692.1,16/02/1994,R,2.0,1708.236510,1609.254684,0.638714
5,1,Doha,Qatar Exxon Mobil Open,2020-01-06,ATP250,Outdoor,Hard,1st Round,3,Kecmanovic M.,...,R,2.0,1736.0,1697.2,20/04/1994,R,2.0,1586.235374,1649.768506,0.409574
6,1,Doha,Qatar Exxon Mobil Open,2020-01-06,ATP250,Outdoor,Hard,1st Round,3,Krajinovic F.,...,R,2.0,1830.1,1797.2,08/01/1995,R,2.0,1766.695869,1621.903554,0.697093
7,1,Doha,Qatar Exxon Mobil Open,2020-01-07,ATP250,Outdoor,Hard,2nd Round,3,Wawrinka S.,...,R,1.0,1850.6,1792.1,12/02/1987,R,2.0,1758.293853,1708.236510,0.571544
8,1,Doha,Qatar Exxon Mobil Open,2020-01-07,ATP250,Outdoor,Hard,1st Round,3,Herbert P.H.,...,R,2.0,1626.7,1496.9,30/09/1992,R,1.0,1597.657455,1465.306119,0.681761
9,1,Doha,Qatar Exxon Mobil Open,2020-01-07,ATP250,Outdoor,Hard,1st Round,3,Kukushkin M.,...,R,2.0,1536.0,1511.3,20/01/1984,R,2.0,1657.733637,1472.203960,0.744215


In [16]:
def simulation(model,
               tournament, 
               features_to_drop=[], 
               missing_values="drop", 
               drop_first=False):
    
    tournament_to_test = unify_data(tournament, features_to_drop, missing_values, drop_first)
    
    prediction = model.predict(tournament_to_test[tournament_to_test['Round'] == 0])
    accuracy = round(accuracy_score(y_true=np.ones(len(prediction), dtype=int), y_pred=prediction), 3)
    print("Accuracy: ", accuracy)
    
    firstR = tournament[tournament['Round'] == '1st Round']
    i = 0
    for j, match in firstR.iterrows():
        print('P1: ' + match['Winner'], 'P2: '+ match['Loser'], 'Wins:', prediction[i])
        i+=1

    #print(prediction)
    
    return tournament_to_test, X, Y
    

In [17]:
T, X, Y = simulation(rf, tournament, features_to_drop)

Accuracy:  0.5
P1: Bublik A. P2: Mannarino A. Wins: 0
P1: Moutet C. P2: Sandgren T. Wins: 0
P1: Verdasco F. P2: Andujar P. Wins: 0
P1: Bedene A. P2: Ymer M. Wins: 1
P1: Chardy J. P2: Barrere G. Wins: 1
P1: Kecmanovic M. P2: Thompson J. Wins: 0
P1: Krajinovic F. P2: Edmund K. Wins: 1
P1: Herbert P.H. P2: Cecchinato M. Wins: 1
P1: Kukushkin M. P2: Jaziri M. Wins: 1
P1: Fucsovics M. P2: Tiafoe F. Wins: 0
P1: Djere L. P2: Sonego L. Wins: 1
P1: Ilkel C. P2: Berankis R. Wins: 0


In [8]:
#print(T.iloc[0],T.iloc[4], T.iloc[7], T.iloc[8])

Notare che il modello sbaglia la predizione quando anche le scommesse(Max e Avg) sbagliano(era più quotata la vittoria del giocatore che di fatto ha perso)
TODO: 
    - fare un decisore stupido basato sulle scommesse
    - provare a tenere B365, nel caso droppare Avg e Max
    
## Preparazione per Australian Open 2020
Scraping dei giocatori del primo round

In [9]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep
from preprocessing import uniformName

In [10]:
columns = {
    'Series': object,
    'Court': object,
    'Surface': object,
    'Round': object,
    'Best of': int,
    'Winner': object,
    'Loser': object,
    'WRank': float,
    'LRank': float,
    'WPts': float,
    'LPts': float,
    'WElo': float,
    'WSurfElo': float,
    'WHand': object,
    'WBHand': float,
    'LElo': float,
    'LSurfElo': float,
    'LHand': object,
    'LBHand': float,
    'WEloCalc': float,
    'LEloCalc': float,
    'ProbaElo': float}
data = {
    'Series': [],
    'Court': [],
    'Surface': [],
    'Round': [],
    'Best of': [],
    'Winner': [],
    'Loser': [],
    'WRank': [],
    'LRank': [],
    'WPts': [],
    'LPts': [],
    'WElo': [],
    'WSurfElo': [],
    'WHand': [],
    'WBHand': [],
    'LElo': [],
    'LSurfElo': [],
    'LHand': [],
    'LBHand': [],
    'WEloCalc': [],
    'LEloCalc': [],
    'ProbaElo': []}

In [11]:
driver = webdriver.Chrome()
driver.get('https://ausopen.com/draws#!mens-singles')
sleep(5)
firstRound = driver.find_element_by_class_name('columm-1')
matches = firstRound.find_elements_by_class_name('match-teams')
i = 1
for m in matches:
    print('Match ', i)
    players = m.find_elements_by_class_name('team-detail__players')
    for player in players:
        try:
            p = player.find_element_by_class_name('player-full-name').get_attribute('innerHTML')
        except Exception:
            p = 'Qualifier'
            
        print(p)
    i+=1

Match  1
Rafael Nadal
Hugo Dellien
Match  2
Federico Delbonis
Joao Sousa
Match  3
Qualifier
Qualifier
Match  4
Qualifier
Pablo Carreno Busta
Match  5
Nick Kyrgios
Lorenzo Sonego
Match  6
Pablo Cuevas
Gilles Simon
Match  7
Yasutaka Uchiyama
Mikael Ymer
Match  8
Qualifier
Karen Khachanov
Match  9
Gael Monfils
Yen-Hsun Lu
Match  10
Ivo Karlovic
Vasek Pospisil
Match  11
James Duckworth
Aljaz Bedene
Match  12
Qualifier
Felix Auger-Aliassime
Match  13
Taylor Fritz
Qualifier
Match  14
Qualifier
Kevin Anderson
Match  15
Alex Bolt
Albert Ramos-Vinolas
Match  16
Adrian Mannarino
Dominic Thiem
Match  17
Daniil Medvedev
Frances Tiafoe
Match  18
Dominik Koepfer
Qualifier
Match  19
Hugo Gaston
Jaume Munar
Match  20
Alexei Popyrin
Jo-Wilfried Tsonga
Match  21
John Isner
Thiago Monteiro
Match  22
Qualifier
Qualifier
Match  23
Miomir Kecmanovic
Andreas Seppi
Match  24
Damir Dzumhur
Stan Wawrinka
Match  25
David Goffin
Jeremy Chardy
Match  26
Pierre-Hugues Herbert
Cameron Norrie
Match  27
Yuichi Sugita
